In [ ]:
using MacroEnergy
using MacroEnergy: get_edges, can_retrofit, is_retrofit, new_capacity, retired_capacity
using Gurobi
using JuMP

system = MacroEnergy.load_system(@__DIR__)

In [ ]:
model = MacroEnergy.generate_model(system)
MacroEnergy.set_optimizer(model, Gurobi.Optimizer)
MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
MacroEnergy.optimize!(model)

In [ ]:
MacroEnergy.compute_conflict!(model)
list_of_conflicting_constraints = MacroEnergy.ConstraintRef[];
for (F, S) in Macro.list_of_constraint_types(model)
    for con in Macro.JuMP.all_constraints(model, F, S)
        if Macro.JuMP.get_attribute(con, Macro.MOI.ConstraintConflictStatus()) == Macro.MOI.IN_CONFLICT
            push!(list_of_conflicting_constraints, con)
        end
    end
end
display(list_of_conflicting_constraints)

In [ ]:
include("../helper_functions.jl")
using DataFrames

In [ ]:
nodes = Node[x for x in system.locations if x isa MacroEnergy.Node]

In [ ]:
co2_nodes = MacroEnergy.get_nodes_sametype(nodes, CO2)

In [ ]:
case = "0pct_co2_reduction"
results_dir = joinpath(@__DIR__,  "results")
mkpath(results_dir)
save_results(results_dir, case, system)

In [ ]:
model[:cRetrofitCapacity]

In [ ]:
function get_unique_retrofit_tuples(system::MacroEnergy.System)
    # Returns a list of retrofit clusters in the system, described as tuples: (retrofit_id, location)
    can_retrofit_ids = []
    is_retrofit_ids = []
    for e in get_edges(system)
        if can_retrofit(e)
            push!(can_retrofit_ids, (e.retrofit_id, e.location))
        end
        if is_retrofit(e)
            push!(is_retrofit_ids, (e.retrofit_id, e.location))
        end
    end
    @assert unique(can_retrofit_ids) == unique(is_retrofit_ids)
    retrofit_ids = unique(can_retrofit_ids)
    return retrofit_ids
end

function get_can_retrofit_edges(edges, retrofit_id, location)
    # Returns the edges that can be retrofitted with a given retrofit_id and location
    can_retrofit_edges = []
    for edge in edges
        if (can_retrofit(edge) == true) && (edge.retrofit_id == retrofit_id) && (edge.location == location)
            push!(can_retrofit_edges, edge)
        end
    end
    return can_retrofit_edges
end

function get_is_retrofit_edges(edges, retrofit_id, location)
    # Returns the edges that can retrofit other edges with a given retrofit_id and location
    is_retrofit_edges = []
    for edge in edges
        if (is_retrofit(edge) == true) && (edge.retrofit_id == retrofit_id) && (edge.location == location)
            push!(is_retrofit_edges, edge)
        end
    end
    return is_retrofit_edges
end

In [ ]:
edges = get_edges(system)
retrofit_tuples = get_unique_retrofit_tuples(system)

@constraint(model, cRetrofitCapacity[(retrofit_id, retrofit_location) in retrofit_tuples],
    sum(new_capacity(e) for e in get_can_retrofit_edges(edges, retrofit_id, retrofit_location)) <=
    sum(retired_capacity(e) for e in get_is_retrofit_edges(edges, retrofit_id, retrofit_location))
)

In [ ]:
edges = get_edges(system)

In [ ]:
nodes = Node[x for x in system.locations if x isa MacroEnergy.Node]
co2_node = MacroEnergy.get_nodes_sametype(nodes, CO2)[1]

In [ ]:
system.assets[1].cement_edge.end_vertex